In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.datasets import fetch_20newsgroups
data=fetch_20newsgroups()
data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [3]:
train=fetch_20newsgroups(subset='train',categories=data.target_names)
test=fetch_20newsgroups(subset='test',categories=data.target_names)

In [4]:
print(len(train.data))

11314


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vector=CountVectorizer()
X_train=vector.fit_transform(train.data)
X_train.shape


(11314, 130107)

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer=TfidfTransformer()
X_train_tfidf=vectorizer.fit_transform(X_train)
X_train_tfidf.shape


(11314, 130107)

In [7]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
clf=BaggingClassifier(KNeighborsClassifier()).fit(X_train_tfidf,train.target)

In [ ]:
X_test=vector.transform(test.data)
X_test_tfidf=vectorizer.transform(X_test)
predicted= clf.predict(X_test_tfidf)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print("accuracy:",accuracy_score(test.target,predicted))
print("confusion matrix:",test.target,predicted)
print(classification_report(test.target,predicted,target_names=test.target_names))

In [ ]:
import nltk
nltk.download('stopwords')
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w).lower() for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_bgc_stemmed = Pipeline([('vect', stemmed_count_vect), ('tfidf', TfidfTransformer()), 
                             ('bgc',BaggingClassifier(KNeighborsClassifier()))])

text_bgc_stemmed = text_bgc_stemmed.fit(train.data, train.target)

predicted_bgc_stemmed = text_bgc_stemmed.predict(test.data)
print("accuracy:",accuracy_score(test.target,predicted_bgc_stemmed))

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
class LemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(LemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([lemmatizer.lemmatize(w).lower() for w in analyzer(doc)])
    
lemmed_count_vect = LemmedCountVectorizer(stop_words='english')

text_bgc_stemmed = Pipeline([('vect', lemmed_count_vect), ('tfidf', TfidfTransformer()), 
                             ('bgc', BaggingClassifier(KNeighborsClassifier()))])

text_bgc_stemmed = text_bgc_stemmed.fit(train.data, train.target)

predicted_bgc_stemmed = text_bgc_stemmed.predict(test.data)
print("accuracy:",accuracy_score(test.target,predicted_bgc_stemmed))